In [203]:
import json
import os, glob

from itertools import product

import numpy as np
import pandas as pd

import torch

import matplotlib.pyplot as plt
%matplotlib notebook

import librosa
import torchaudio

from torchaudio.functional.filtering import lowpass_biquad, highpass_biquad
from torchaudio.transforms import Spectrogram, AmplitudeToDB, MelSpectrogram, Resample
from synth.synth_constants import synth_constants
from synth.synth_modules import ADSR


from matplotlib import rcParams

from collections import defaultdict

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import seaborn as sns

from synth.synth_architecture import SynthModular

from IPython.display import Audio, display
from ipywidgets import interactive, interact_manual, FloatSlider, interact
import ipywidgets as widgets

sns.set_style('whitegrid')

n_fft = 1024
win_length = None
hop_length = 512

# Define transform
spectrogram = Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)

n_mels = 20

mel_spectrogram = MelSpectrogram(
    sample_rate=synth_constants.sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    f_min=0,
    f_max=20,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm="slaney",
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

resample = Resample(16000, 50)

# spectrogram = Spectrogram(n_fft=1024)
db = AmplitudeToDB(stype='magnitude')

def plot_spectrogram(specgram, figure_id = 0, title=None, ylabel="freq_bin", db=True):
    fig = plt.figure(figure_id, figsize=(13, 2.5))
    axs = fig.add_subplot(1, 1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    if db:
        im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", cmap='inferno')
    else:
        im = axs.imshow(specgram, origin="lower", aspect="auto", cmap='inferno')
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

signal_duration = 1
device = 'cuda:0'
batch_size=1


C:\Users\noamk\PycharmProjects\ai_synth\venv\lib\site-packages\torchaudio\functional\functional.py:507: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (20) may be set too high. Or, the value for `n_freqs` (513) may be set too low.
  warnings.warn(


In [3]:
semitones_list = [*range(-24, 24 + 1)]
middle_c_freq= 261.6255653005985
osc_freq_list = [middle_c_freq * (2 ** (1 / 12)) ** x for x in semitones_list]
osc_freq_list

[65.40639132514956,
 69.29565774421792,
 73.41619197935178,
 77.78174593052012,
 82.40688922821737,
 87.30705785825086,
 92.49860567790849,
 97.9988589954372,
 103.82617439498617,
 109.99999999999989,
 116.54094037952234,
 123.4708253140309,
 130.81278265029917,
 138.59131548843592,
 146.83238395870364,
 155.56349186104032,
 164.81377845643482,
 174.6141157165018,
 184.99721135581706,
 195.9977179908745,
 207.65234878997242,
 219.99999999999986,
 233.08188075904485,
 246.94165062806192,
 261.6255653005985,
 277.18263097687196,
 293.6647679174074,
 311.1269837220808,
 329.62755691286986,
 349.22823143300377,
 369.9944227116343,
 391.9954359817492,
 415.30469757994507,
 439.99999999999994,
 466.1637615180899,
 493.8833012561241,
 523.2511306011972,
 554.3652619537442,
 587.3295358348153,
 622.253967444162,
 659.2551138257401,
 698.4564628660079,
 739.9888454232689,
 783.9908719634988,
 830.6093951598906,
 880.0000000000003,
 932.3275230361803,
 987.7666025122487,
 1046.5022612023952]

In [3]:
%matplotlib notebook


In [100]:
synth_obj = SynthModular(preset_name='MODULAR_NEW', synth_constants=synth_constants, device='cpu')

params = {(0,0): {'operation': 'lfo_sine', 'parameters': {'freq': [10], 'active': [True], 'output': [[1, 1]]}},
          (1,1): {'operation': 'fm_lfo', 'parameters': {'freq_c': [3], 'waveform': ['sine'], 'fm_lfo_mod_index': [0.005], 'amp_c': [1], 'active': [True], 'fm_active': [True], 'output': [[1, 2]]}},
          (0,2): {'operation': 'fm_sine', 'parameters': {'amp_c': [1], 'freq_c': [200], 'mod_index': [0.1], 'active': [False], 'fm_active': [False]}},
          (1,2): {'operation': 'fm_saw', 'parameters': {'amp_c': [1], 'freq_c': [750], 'mod_index': [0.1], 'active': [True], 'fm_active': [True]}},
          (2,2): {'operation': 'fm_square', 'parameters': {'amp_c': [1], 'freq_c': [1000], 'mod_index': [0.1], 'active': [False], 'fm_active': [False]}},
          (0,3): {'operation': 'mix', 'parameters': {}},
          (0,4): {'operation': 'env_adsr', 'parameters': {'attack_t': [0], 'decay_t': [0], 'sustain_t': [3], 'sustain_level': [0.5], 'release_t': [1]}},
          (0,5): {'operation': 'lowpass_filter_adsr', 'parameters': {'attack_t': [1], 'decay_t': [1], 'sustain_t': [1], 'sustain_level': [0.1], 'release_t': [1], 'filter_freq': [0], 'intensity': [1]}},
          (0,6): {'operation': 'tremolo', 'parameters': {'amount': [0.4], 'active': [True], 'fm_active': [False]}}
          }

synth_obj.update_cells_from_dict(params)
final_signal, output_signals_through_chain_dict = synth_obj.generate_signal(signal_duration=signal_duration, batch_size=1)
sound = final_signal.detach().cpu()[0]

spec = spectrogram(sound)
db_spec = db(spec)
melspec = mel_spectrogram(sound.float())

audio = Audio(sound.numpy(), rate=synth_constants.sample_rate, autoplay=False)
display(audio)

plt.close('all')

fig, ax = plt.subplots(1, 1, figsize=(13, 2.5))
ax.plot(sound.numpy())

plot_spectrogram(spec, title="Spectrogram", db=False)
plot_spectrogram(spec, title="Spectrogram - dB")
plot_spectrogram(melspec, title="MelSpectrogram - torchaudio", ylabel="mel freq")


# Signal Though chain
chain_position = '(1, 1)'

chain_output = output_signals_through_chain_dict[chain_position]
chain_sound = chain_output.detach().cpu()[0]

chain_spec = spectrogram(chain_sound)
chain_db_spec = db(chain_spec)
chain_melspec = mel_spectrogram(chain_sound.float())

chain_audio = Audio(chain_sound.numpy(), rate=synth_constants.sample_rate, autoplay=False)
display(chain_audio)

fig3, ax3 = plt.subplots(1, 1, figsize=(13, 2.5))
ax3.plot(chain_sound.numpy())
plt.title(r"signal in {chain_position}")

plot_spectrogram(chain_spec, title=f"Spectrogram {chain_position}", db=False)
plot_spectrogram(chain_spec, title=f"Spectrogram - dB {chain_position}")
plot_spectrogram(chain_melspec, title=f"MelSpectrogram - {chain_position}", ylabel="mel freq")


# NOTES:
# debug filter adsr - debug different timings and watch for result
# show signals through chain

ValueError: Illegal parameter for the provided operation.

In [6]:
synth_obj = SynthModular(preset_name='MODULAR_NEW', synth_constants=synth_constants, device='cpu')

params = {(0,0): {'operation': 'lfo_sine', 'parameters': {'freq': [20], 'active': [True], 'output': [[1, 1]]}},
          (1,1): {'operation': 'fm_lfo', 'parameters': {'freq_c': [20], 'waveform': ['sine'], 'fm_lfo_mod_index': [0.005], 'active': [True], 'fm_active': [True], 'output': [[1, 2]]}},
          (0,2): {'operation': 'fm_sine', 'parameters': {'amp_c': [1], 'freq_c': [200], 'mod_index': [0.1], 'active': [False], 'fm_active': [False]}},
          (1,2): {'operation': 'fm_saw', 'parameters': {'amp_c': [1], 'freq_c': [750], 'mod_index': [0.1], 'active': [True], 'fm_active': [True]}},
          (2,2): {'operation': 'fm_square', 'parameters': {'amp_c': [1], 'freq_c': [1000], 'mod_index': [0.1], 'active': [False], 'fm_active': [False]}},
          (0,3): {'operation': 'mix', 'parameters': {}},
          (0,4): {'operation': 'env_adsr', 'parameters': {'attack_t': [0], 'decay_t': [0], 'sustain_t': [3], 'sustain_level': [0.5], 'release_t': [1]}},
          (0,5): {'operation': 'lowpass_filter_adsr', 'parameters': {'attack_t': [1], 'decay_t': [1], 'sustain_t': [1], 'sustain_level': [0.1], 'release_t': [1], 'filter_freq': [0], 'intensity': [1]}},
          (0,6): {'operation': 'tremolo', 'parameters': {'amount': [0.4], 'active': [True], 'fm_active': [False]}}
          }

synth_obj.update_cells_from_dict(params)
final_signal, output_signals_through_chain_dict = synth_obj.generate_signal(signal_duration=signal_duration, batch_size=1)
sound = final_signal.detach().cpu()[0]

spec = spectrogram(sound)
db_spec = db(spec)
melspec = mel_spectrogram(sound.float())

audio = Audio(sound.numpy(), rate=synth_constants.sample_rate, autoplay=False)
display(audio)

plt.close('all')

# fig, ax = plt.subplots(1, 1, figsize=(13, 2.5))
# ax.plot(sound.numpy())
#
# plot_spectrogram(spec, title="Spectrogram", db=False)
# plot_spectrogram(spec, title="Spectrogram - dB")
# plot_spectrogram(melspec, title="MelSpectrogram - torchaudio", ylabel="mel freq")


# Signal Though chain
chain_position = '(1, 1)'

chain_output = output_signals_through_chain_dict[chain_position]
chain_sound = chain_output.detach().cpu()[0]

chain_spec = spectrogram(chain_sound)
chain_db_spec = db(chain_spec)
chain_melspec = mel_spectrogram(chain_sound.float())

chain_audio = Audio(chain_sound.numpy(), rate=synth_constants.sample_rate, autoplay=False)
display(chain_audio)

fig3, ax3 = plt.subplots(1, 1, figsize=(13, 2.5))
ax3.plot(chain_sound.numpy())
plt.title(r"signal in {chain_position}")

plot_spectrogram(chain_spec, title=f"Spectrogram {chain_position}", db=False)
plot_spectrogram(chain_spec, title=f"Spectrogram - dB {chain_position}")
plot_spectrogram(chain_melspec, title=f"MelSpectrogram - {chain_position}", ylabel="mel freq")

resampled = resample(chain_sound)

plot_spectrogram(resampled_spec, title=f"Spectrogram - dB {chain_position}")


# NOTES:
# debug filter adsr - debug different timings and watch for result
# show signals through chain

Missing amp param in Oscillator module lfo_sine. Assuming fixed amp. Please check Synth structure if this is unexpected.
Missing amp param in Oscillator module fm_lfo. Assuming fixed amp. Please check Synth structure if this is unexpected.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [229]:
synth_obj = SynthModular(preset_name='LFO_SAW', synth_constants=synth_constants, device='cpu')

params = {
          (1,1): {'operation': 'lfo', 'parameters': {'freq': [10], 'waveform': ['square'], 'active': [True],
                                                     'output': [[0, 2]]}},
          (0,2): {'operation': 'fm_saw', 'parameters': {'amp_c': [1], 'freq_c': [200], 'mod_index': [0.1], 'active': [True], 'fm_active': [True]}},
          }

synth_obj.update_cells_from_dict(params)
final_signal, output_signals_through_chain_dict = synth_obj.generate_signal(signal_duration=signal_duration, batch_size=1)
sound = final_signal.detach().cpu()[0]

n_fft = 1024
win_length = None
hop_length = 512

spectrogram = Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)

lfo_spectrogram = Spectrogram(
    n_fft=128,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)

n_mels = 128

mel_spectrogram = MelSpectrogram(
    sample_rate=synth_constants.sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    f_min=0,
    f_max=8000,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm="slaney",
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

lfo_mel_spectrogram = MelSpectrogram(
    sample_rate=synth_constants.sample_rate,
    n_fft=128,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm="slaney",
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

resample = Resample(16000, 100)


# spec = spectrogram(sound)
# db_spec = db(spec)
# melspec = mel_spectrogram(sound.float())
#
# audio = Audio(sound.numpy(), rate=synth_constants.sample_rate, autoplay=False)
# display(audio)

plt.close('all')

# fig, ax = plt.subplots(1, 1, figsize=(13, 2.5))
# fig = plt.figure(0, figsize=(13, 2.5))
# ax = fig.add_subplot(1, 1, 1)
# ax.set_title("final signal in time")
# ax.plot(sound.numpy())
# plt.show()
#
# plot_spectrogram(spec, figure_id=1, title="Spectrogram", db=False)
# plot_spectrogram(spec, figure_id=2, title="Spectrogram - dB")
# plot_spectrogram(melspec, figure_id=3, title="MelSpectrogram - torchaudio", ylabel="mel freq")


# # Signal Though chain
chain_position = '(1, 1)'

chain_output = output_signals_through_chain_dict[chain_position]
chain_sound = chain_output.detach().cpu()[0]

# fig1 = plt.figure(4, figsize=(13, 2.5))
# ax1 = fig1.add_subplot(1, 1, 1)
# ax1.plot(chain_sound.numpy())
# plt.title(f"signal in {chain_position}")
# plt.show()

chain_sound = resample(chain_sound)

chain_spec = lfo_spectrogram(chain_sound)
chain_db_spec = db(chain_sound)
chain_melspec = lfo_mel_spectrogram(chain_sound.float())

chain_audio = Audio(chain_sound.numpy(), rate=synth_constants.sample_rate, autoplay=False)
display(chain_audio)

fig2 = plt.figure(4, figsize=(13, 2.5))
ax2 = fig2.add_subplot(1, 1, 1)
ax2.plot(chain_sound.numpy())
plt.title(f"signal in {chain_position}")
plt.show()

plot_spectrogram(chain_spec, figure_id=6, title=f"Spectrogram {chain_position}", db=False)
plot_spectrogram(chain_spec, figure_id=7, title=f"Spectrogram - dB {chain_position}")
plot_spectrogram(chain_melspec, figure_id=8, title=f"MelSpectrogram - {chain_position}", ylabel="mel freq")
#

# resampled_spec = lfo_spectrogram(resampled)

# plot_spectrogram(resampled_spec, title=f"Spectrogram - dB {chain_position}")


# NOTES:
# debug filter adsr - debug different timings and watch for result
# show signals through chain

Missing amp param in Oscillator module lfo. Assuming fixed amp. Please check Synth structure if this is unexpected.


C:\Users\noamk\PycharmProjects\ai_synth\venv\lib\site-packages\torchaudio\functional\functional.py:507: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (65) may be set too low.
  warnings.warn(


KeyboardInterrupt: 

In [228]:
# chain_sound.shape
chain_db_spec.shape

torch.Size([100])

In [227]:
chain_spec.shape

torch.Size([65, 1])

In [8]:
window = torch.hann_window(1024, requires_grad=True)
zeros = torch.zeros(512)

a = torch.ones(2048)
b = a.unfold(dimension=0, size=1024, step=512)
filtered = b * window

f1 = torch.cat((filtered[0], torch.zeros(512*2)))
f2 = torch.cat((torch.zeros(512), filtered[1], torch.zeros(512)))
f3 = torch.cat((torch.zeros(512*2), filtered[2]))
# second = torch.cat((zeros, window, zeros))
# third = torch.cat((zeros, zeros, window))
#
full = f1 + f2 + f3

fig, ax = plt.subplots(1, 1, figsize=(20, 5))
ax.plot(full.detach().cpu().numpy())


<IPython.core.display.Javascript object>